In [1]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Mean & Std Dev of LST
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        uhs = lst_c.gt(threshold).rename('UHS') # gt() produces a Byte/Int image
        # Return a NEW image with ONLY the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # === START OF FIX ===
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # === END OF FIX ===
        
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # === START OF FIX ===
    # The order of .mean() and .select() has been swapped.
    image_composite = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds)
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()  # First, create a single mean composite image.
        .select(index_name)) # Then, select the desired band from that composite.
    # === END OF FIX ===

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Processing... Please wait."

    try:
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return

        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return

        selected_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
        analysis_collection = FLORIDA_COUNTIES if selected_geo_name == 'Florida' else selected_geometry

        status_label.value = f"Status: Calculating {selected_index}..."
        ee_years = ee.List(years_to_process)

        def get_stats_for_year(year):
            mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
            # The scale here should be appropriate for the analysis scale
            return mean_image.reduceRegions(
                collection=analysis_collection, reducer=ee.Reducer.mean(), scale=200
            ).map(lambda f: f.set('year', year))

        all_stats_info = ee.FeatureCollection(ee_years.map(get_stats_for_year)).flatten().getInfo()['features']

        status_label.value = "Status: Aggregating results..."
        df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                   for f in all_stats_info if f['properties'].get('mean') is not None]

        if not df_data:
            status_label.value = "Status: No data found for the selected criteria."
            with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>Try expanding the date range or selecting a different area.</p>"))
            return

        df = pd.DataFrame(df_data).dropna()
        df[selected_index] = df[selected_index].round(4)

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            m.centerObject(selected_geometry, 9 if selected_geo_name != 'Florida' else 7)
            
            if selected_geo_name == 'Florida':
                # Use a join to efficiently color the counties based on the dataframe
                for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if year_df.empty:
                        continue
                    
                    # Create a dictionary for mapping county names to values
                    county_data_map = year_df.set_index('County')[selected_index].to_dict()
                    
                    # Map over the counties feature collection to add the value as a property
                    def add_data_to_fc(feature):
                        county_name = feature.get('NAME')
                        value = ee.Number(county_data_map.get(county_name))
                        return feature.set('vis_value', value)
                        
                    counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                    
                    # Paint the counties using the 'vis_value' property
                    image_to_display = ee.Image().toFloat().paint(counties_with_data, 'vis_value')
                    m.addLayer(image_to_display.clip(FLORIDA_STATE), vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            else:
                for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            
            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            # Pivot the dataframe for clear presentation
            pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
            display(pivot_df)

        with rank_output:
            # Calculate mean values only if the analysis was for the entire state
            if selected_geo_name == 'Florida':
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                rank_html = f"""
                <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                <div style="display: flex; justify-content: space-around;">
                    <div><h4>{vis_config['rank_high']}</h4>{mean_values.nlargest(5).to_frame().to_html(header=False)}</div>
                    <div><h4>{vis_config['rank_low']}</h4>{mean_values.nsmallest(5).to_frame().to_html(header=False)}</div>
                </div>"""
                display(widgets.HTML(rank_html))
            else:
                 display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

        status_label.value = "Status: Done."

    except ee.EEException as e:
        status_label.value = f"A GEE error occurred: {e}"
        with map_output:
             display(widgets.HTML(f"<h3>Google Earth Engine Error:</h3><p>The server returned an error. This can happen with very large or long requests.</p><pre>{e}</pre>"))
    except Exception as e:
        import traceback
        status_label.value = f"An unexpected error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Application Error:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [ ]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Mean & Std Dev of LST
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        
        # --- FIX IS HERE ---
        # Explicitly cast the result of .gt() to a Byte to ensure type consistency.
        uhs = lst_c.gt(threshold).toByte().rename('UHS') 
        
        # Return a NEW image with all the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # The order of .mean() and .select() has been swapped for efficiency.
    image_composite = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds)
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()  # First, create a single mean composite image.
        .select(index_name)) # Then, select the desired band from that composite.

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# 6. EVENT HANDLER (MODIFIED FOR BATCH PROCESSING)
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        # --- User Selections ---
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        # --- Input Validation ---
        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return
        
        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return
            
        # --- NEW: Batch Processing Logic ---
        all_df_data = [] # A list to accumulate results from all batches
        
        # We only need batching for the statewide analysis
        if selected_geo_name == 'Florida':
            batch_size = 3 # Process 3 counties at a time
            county_fc_list = FLORIDA_COUNTIES.toList(FLORIDA_COUNTIES.size())
            num_counties = county_fc_list.size().getInfo()

            for i in range(0, num_counties, batch_size):
                status_label.value = f"Status: Processing counties {i+1} to {min(i+batch_size, num_counties)} of {num_counties}..."
                
                # Get a small batch of counties to process
                county_batch = ee.FeatureCollection(county_fc_list.slice(i, i + batch_size))
                
                # Define a function to get stats for the batch
                def get_stats_for_year_batch(year):
                    # Create the composite image for the whole state
                    mean_image = get_mean_index_for_year(year, month_range, FLORIDA_STATE, selected_index)
                    # Reduce regions for ONLY the current batch of counties
                    return mean_image.reduceRegions(
                        collection=county_batch, reducer=ee.Reducer.mean(), scale=1000 # Using a larger scale
                    ).map(lambda f: f.set('year', year))
                
                # Run the analysis for the current batch and get the results
                batch_stats_info = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year_batch)).flatten().getInfo()['features']
                
                # Append results from this batch to our main list
                batch_df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                                 for f in batch_stats_info if f['properties'].get('mean') is not None]
                all_df_data.extend(batch_df_data)

            # Use the aggregated data from all batches
            df_data = all_df_data
        
        else: # Original logic for single-county analysis
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
            
            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                return mean_image.reduceRegions(
                    collection=selected_geometry, reducer=ee.Reducer.mean(), scale=200
                ).map(lambda f: f.set('year', year))
                
            all_stats_info = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year)).flatten().getInfo()['features']
            df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                       for f in all_stats_info if f['properties'].get('mean') is not None]

        # --- Display Results (This part remains the same) ---
        status_label.value = "Status: Aggregating and displaying results..."
        if not df_data:
            status_label.value = "Status: No data found for the selected criteria."
            with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>Try expanding the date range or selecting a different area.</p>"))
            return

        df = pd.DataFrame(df_data).dropna()
        df[selected_index] = df[selected_index].round(4)

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            display_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else selected_geometry
            m.centerObject(display_geometry, 7 if selected_geo_name == 'Florida' else 9)
            
            if selected_geo_name == 'Florida':
                for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if not year_df.empty:
                        county_data_map = year_df.set_index('County')[selected_index].to_dict()
                        
                        def add_data_to_fc(feature):
                            county_name = feature.get('NAME')
                            value = ee.Number(county_data_map.get(county_name))
                            return feature.set('vis_value', value)
                            
                        counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                        image_to_display = ee.Image().toFloat().paint(counties_with_data, 'vis_value')
                        m.addLayer(image_to_display.clip(FLORIDA_STATE), vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            else: # Single county raster display
                 for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))

            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
            display(pivot_df)

        with rank_output:
            if selected_geo_name == 'Florida':
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                vis_config = VIS_PARAMS[selected_index]
                rank_html = f"""
                <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                <div style="display: flex; justify-content: space-around;">
                    <div><h4>{vis_config['rank_high']}</h4>{mean_values.nlargest(5).to_frame().to_html(header=False)}</div>
                    <div><h4>{vis_config['rank_low']}</h4>{mean_values.nsmallest(5).to_frame().to_html(header=False)}</div>
                </div>"""
                display(widgets.HTML(rank_html))
            else:
                display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

        status_label.value = "Status: Done."

    except ee.EEException as e:
        status_label.value = f"A GEE error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Google Earth Engine Error:</h3><p>The server returned an error. This can happen with very large or long requests.</p><pre>{e}</pre>"))
    except Exception as e:
        import traceback
        status_label.value = f"An unexpected error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Application Error:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')